In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch torchvision
!pip install yolov5

In [49]:
import torch
from pycocotools.coco import COCO
import os
import cv2
import json
from pycocotools.cocoeval import COCOeval

# Define categories
thing_classes = ['boat']

def run_evaluation(model, img_dir, json_file, categories):
    coco_api = COCO(json_file)
    cat_ids = coco_api.getCatIds(catNms=categories)
    img_ids = coco_api.getImgIds(catIds=cat_ids)
    predictions = []

    for img_id in img_ids:
        img_info = coco_api.loadImgs(img_id)[0]
        img_path = os.path.join(img_dir, img_info['file_name'])
        try:
            image = cv2.imread(img_path)
            if image is not None:
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = model(image_rgb)
                for result in results.pandas().xyxy[0].to_dict(orient='records'):
                    predictions.append({
                        "image_id": img_id,
                        "category_id": cat_ids[0],
                        "bbox": [result['xmin'], result['ymin'], result['xmax'] - result['xmin'], result['ymax'] - result['ymin']],
                        "score": result['confidence']
                    })
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

    # Save prediction results as JSON file
    prediction_file = os.path.join(img_dir, 'predictions.json')
    with open(prediction_file, 'w') as f:
        json.dump(predictions, f)

    # Load prediction results and ground truth values for evaluation
    coco_gt = COCO(json_file)
    coco_dt = coco_gt.loadRes(prediction_file)
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

# Model loading
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

# Specify folder path
train_img_dir = "/content/drive/MyDrive/train"
train_json = "/content/drive/MyDrive/train.json"
val_img_dir = "/content/drive/MyDrive/val"
val_json = "/content/drive/MyDrive/val.json"

print("Training...")
run_evaluation(model, train_img_dir, train_json, thing_classes)
print("Validation...")
run_evaluation(model, val_img_dir, val_json, thing_classes)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 v7.0-313-g712de55a Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 


Training...
loading annotations into memory...
Done (t=0.23s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.393
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.556
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.451
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.418
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.309
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=

In [7]:
import torch
from pycocotools.coco import COCO
import os
import cv2
import json
from pycocotools.cocoeval import COCOeval

# Define categories
thing_classes = ['boat']

def run_evaluation(model, img_dir, json_file, categories, size_threshold=64):
    coco_api = COCO(json_file)
    cat_ids = coco_api.getCatIds(catNms=categories)
    img_ids = coco_api.getImgIds(catIds=cat_ids)
    predictions = []

    for img_id in img_ids:
        img_info = coco_api.loadImgs(img_id)[0]
        img_path = os.path.join(img_dir, img_info['file_name'])
        try:
            image = cv2.imread(img_path)
            if image is not None:
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = model(image_rgb)
                for result in results.pandas().xyxy[0].to_dict(orient='records'):
                    bbox_width = result['xmax'] - result['xmin']
                    bbox_height = result['ymax'] - result['ymin']
                    # Ignore small objects
                    if bbox_width > size_threshold and bbox_height > size_threshold:
                        predictions.append({
                            "image_id": img_id,
                            "category_id": cat_ids[0],
                            "bbox": [result['xmin'], result['ymin'], bbox_width, bbox_height],
                            "score": result['confidence']
                        })
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

    # Save prediction results as JSON file
    prediction_file = os.path.join(img_dir, 'predictions.json')
    with open(prediction_file, 'w') as f:
        json.dump(predictions, f)

    # Load prediction results and ground truth values for evaluation
    coco_gt = COCO(json_file)
    coco_dt = coco_gt.loadRes(prediction_file)
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

# Model loading
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

# Specify folder path
train_img_dir = "/content/drive/MyDrive/train"
train_json = "/content/drive/MyDrive/train.json"
val_img_dir = "/content/drive/MyDrive/val"
val_json = "/content/drive/MyDrive/val.json"

print("Training...")
run_evaluation(model, train_img_dir, train_json, thing_classes)
print("Validation...")
run_evaluation(model, val_img_dir, val_json, thing_classes)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-28 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 


Training...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.400
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.550
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.469
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.416
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.327
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=